In [61]:
# basic python and ML Libraries
import os
import numpy as np
import pandas as pd
import pathlib 
import re

# matplotlib & others for visualization
import matplotlib.pyplot as plt
from PIL import Image, ImageDraw
import cv2

# torchvision libraries
import torch
from torch import nn
import torchvision
from torchvision import transforms
from torch.utils.data import DataLoader, Dataset

# misc
from tqdm import tqdm
from evaluate import load
from transformers import AdamW
from wordsegment import load, segment

# model
from strhub.data.module import SceneTextDataModule

device = "cuda" if torch.cuda.is_available() else "cpu"
working_dir = r'C:\Users\earle\PythonMLenv\env'
os.getcwd()
load()

In [ ]:
# functions
def ToDf(ostlistpath , label):
    file_name = []
    text = []
    for i in ostlistpath:
        file_name.append(i)
        with open(label + i[:-4] + '.txt') as f:
            line = [line for line in f][0]
            text.append(line)

    d = {'file_name': file_name, 'text' : text}
    df = pd.DataFrame(d)
    return df



### parseq torchhub 

In [29]:
# git clone https://github.com/baudm/parseq
# os.chdir(r'C:\Users\earle\PythonMLenv\parseq')
train_img_dir = r'C:\Users\earle\PythonMLenv\env\projects\data\Higher_lower\\train\\images\\'
train_img = [train_img_dir + i for i in os.listdir(train_img_dir)]
parseq = torch.hub.load('baudm/parseq', 'parseq', pretrained= True).eval()
img_transform = SceneTextDataModule.get_transform(parseq.hparams.img_size)

w = []

for i in train_img:
    img = Image.open(i).convert("RGB")
    #display(img)
    img = img_transform(img).unsqueeze(0)
    logits = parseq(img)
    pred = logits.softmax(-1)
    label, confidence = parseq.tokenizer.decode(pred)
    #print(label[0])
    w.append(label[0])



Using cache found in C:\Users\earle/.cache\torch\hub\baudm_parseq_main


In [69]:
l = [' '.join(segment(i)) for i in w]


### hugging face

In [ ]:
# # setups
# train_img_dir = r'C:\Users\earle\PythonMLenv\env\projects\data\Higher_lower\\train\\images\\'
# test_img_dir = r'C:\Users\earle\PythonMLenv\env\projects\data\Higher_lower\\test\\images\\'
# train_label_dir = r'C:\Users\earle\PythonMLenv\env\projects\data\Higher_lower\\train\\label\\'
# test_label_dir = r'C:\Users\earle\PythonMLenv\env\projects\data\Higher_lower\\test\\label\\'
# plist = os.listdir(train_img_dir)
# dict_train = {}
# dict_test = {}

# # for i in plist:
# #     file_name = i[:-4]
# #     image = Image.open(test_img_dir + '\\' + i)
# #     display(image)
# #     with open(test_label_dir + file_name + '.txt', 'w') as file:
# #         image_name_input = input("Enter image text: ")
# #         file.write(image_name_input)

# train_df = ToDf(plist, train_label_dir)
# test_df = ToDf(os.listdir(test_img_dir), test_label_dir)
# train_df.head()

In [ ]:
# # dataset & dataloader

# class HigherLowerDataset(Dataset):
#     def __init__(self, root_dir, df, processor, max_target_length =128) -> None:
#         self.root_dir = root_dir
#         self.df = df
#         self.processor = processor
#         self.max_target_length = max_target_length
    
#     def __getitem__(self, index):
#         file_name = self.df['file_name'][index]
#         text = self.df['text'][index]
#         image = Image.open(self.root_dir + file_name).convert("RGB")
        
#         pixel_values = self.processor(image, return_tensors = 'pt').pixel_values
#         labels = self.processor.tokenizer(text , padding = 'max_length', max_length = self.max_target_length).input_ids
#         labels = [label if label != self.processor.tokenizer.pad_token_id else -100 for label in labels]
        
#         encoding = {"pixel_values": pixel_values.squeeze(), "labels": torch.tensor(labels)}
#         return encoding
    
#     def __len__(self):
#         return len(self.df)
        
# processor = TrOCRProcessor.from_pretrained("microsoft/trocr-base-printed")
# train_dataset = HigherLowerDataset(root_dir=train_img_dir, df = train_df, processor=processor)
# test_dataset = HigherLowerDataset(root_dir=test_img_dir, df = test_df, processor=processor)
# train_dataloader = DataLoader(train_dataset, batch_size=8 , shuffle=True)
# test_dataloader = DataLoader(test_dataset, batch_size=8)

In [ ]:
# # model
# model = VisionEncoderDecoderModel.from_pretrained("microsoft/trocr-base-printed")
# model.to(device)
# model.config.decoder_start_token_id = processor.tokenizer.cls_token_id
# model.config.pad_token_id = processor.tokenizer.pad_token_type_id
# model.config.vocab_size = model.config.decoder.vocab_size

# # set beam search parameters
# model.config.eos_token_id = processor.tokenizer.sep_token_id
# model.config.max_length = 64
# model.config.early_stopping = True
# model.config.no_repeat_ngram_size = 3
# model.config.length_penalty = 2.0
# model.config.num_beams = 4

In [ ]:
# cer_metric = load("cer")

# def compute_cer(pred):
#     labels_ids = pred.label_ids
#     pred_ids = pred.predictions

#     pred_str = processor.batch_decode(pred_ids, skip_special_tokens=True)
#     labels_ids[labels_ids == -100] = processor.tokenizer.pad_token_id
#     label_str = processor.batch_decode(labels_ids, skip_special_tokens=True)
#     cer = cer_metric.compute(predictions=pred_str, references=label_str)
#     return {"cer": cer}

In [ ]:
# train 
# training_args = Seq2SeqTrainingArguments(
#     predict_with_generate=True,
#     evaluation_strategy="steps",
#     per_device_eval_batch_size=1,
#     per_device_train_batch_size=1,
#     fp16=True,
#     output_dir="./",
#     logging_steps=2,
#     save_steps=1000,
#     eval_steps=200,
# )
# trainer = Seq2SeqTrainer(
#     model = model,
#     tokenizer = processor.feature_extractor,
#     args = training_args,
#     compute_metrics=compute_cer,
#     train_dataset= train_dataset,
#     eval_dataset= test_dataset,
#     data_collator=default_data_collator
# )
# trainer.train()
# trainer.save_model(r'C:\Users\earle\PythonMLenv\env\projects\Models')

# train model
optimizer = AdamW(model.parameters(), lr = 5e-5)
optimizer.zero_grad()
coutn = 0
for i in range(3):
    model.train()
    train_loss = 0
    for batch in tqdm(train_dataloader):
        coutn +=1
        for k,v in batch.items():
            batch[k] = v.to(device)
        outputs = model(**batch)
        loss = outputs.loss
        loss.backward()

        if (coutn+1) % 10 ==0:
            optimizer.step()
            optimizer.zero_grad()

        train_loss += loss.item()

    print(f"Loss: {train_loss/len(train_dataloader)}")

    model.eval()
    valid_cer = 0
    with torch.inference_mode():
        for batch in tqdm(test_dataloader):
            outputs = model.generate(batch["pixel_values"]).to(device)
            cer = compute_cer(outputs, batch["labels"])
            valid_cer +=cer
        print("Test CER: ", valid_cer/len(test_dataloader))

In [ ]:
# inference
# processor = TrOCRProcessor.from_pretrained("microsoft/trocr-base-printed")
# model = VisionEncoderDecoderModel.from_pretrained("microsoft/trocr-base-printed")

# val_dir = r'C:\Users\earle\PythonMLenv\env\projects\data\Higher_lower\val\images'
# pjoined = [val_dir + "\\" +i for i in os.listdir(val_dir)]

# for i in pjoined:
#     image = Image.open(i).convert("RGB")
#     display(image)
#     pixel_values = processor(image, return_tensors = "pt").pixel_values
#     generated_ids = model.generate(pixel_values)
#     generated_text = processor.batch_decode(generated_ids,skip_special_tokens = True)[0]
#     print(generated_text)